# Prompt & Context Engineering — Cheat Sheet (with examples)

## Goals

* **Be reliable:** same input → same style/format.
* **Be grounded:** answer only from provided context; refuse when missing.
* **Be controllable:** tone, length, structure, language.
* **Be efficient:** fit token budget, minimize latency/cost.

---

## Core Principles

* **Instruction hierarchy:** system > developer > user > data/context > examples.
* **Single source of truth:** always state scope + allowed sources.
* **Determinism first:** start with low temperature (0–0.3); raise only if needed.
* **Tight feedback loop:** iterate with a small eval set and keep a changelog.
* **Guardrails beat band-aids:** require “I don’t know” behavior explicitly.

---

## Prompt Skeletons (drop-in)

### 1) RAG Answerer (safe, concise)

```
ROLE: You are an assistant that answers ONLY using the provided context.

CONTEXT:
{{context}}

TASK:
Answer the QUESTION strictly using the CONTEXT. If the answer is not present, reply exactly: "I don’t know."

QUESTION:
{{question}}

CONSTRAINTS:
- Be concise (≤ 120 words).
- Cite snippet IDs from CONTEXT if available (e.g., [D3], [P2]).
- No speculation or external facts.
```

### 2) Persona + Style

```
ROLE: You are {{persona}}. Tone: {{tone}}. Audience: {{audience}}.

TASK:
Respond to the user while preserving the persona and tone.

HARD RULES:
- Stay in scope: {{scope}}.
- If out of scope, say: "Sorry, I can only discuss {{scope}}."
- Keep responses under {{max_words}} words unless asked otherwise.
```

### 3) Format-Constrained (JSON)

```
ROLE: Output MUST be valid JSON. No extra text.

SCHEMA:
{"answer": string, "confidence": "high|medium|low", "citations": [string]}

CONTEXT:
{{context}}

QUESTION:
{{question}}

RULES:
- If unsure, answer "" and set "confidence": "low".
- Do not include keys not in the schema.
```

### 4) Structured Reasoning (silent CoT)

```
ROLE: Solve carefully using internal reasoning, but only output the final answer.

INTERNAL STEPS (do not reveal): analyze → plan → verify.

QUESTION:
{{question}}

FINAL OUTPUT (no steps, no notes):
```

### 5) Multi-turn Memory Prompt

```
ROLE: Continue the conversation using the last N turns for context.

CHAT HISTORY (most recent last):
{{history}}

NEW USER MESSAGE:
{{message}}

RULES:
- Prefer recent history when conflicting.
- Summarize long history if needed before answering.
```

---

## Prompt Patterns (when to use & mini-examples)

* **Zero-shot:** quickest baseline.
  “Summarize this in 3 bullets.”

* **Few-shot:** stabilize style/format.
  Provide 2–3 input→output pairs before the task.

* **Self-ask / self-query:** let the model generate subquestions before answering (implicit, no tools).
  “Before answering, list the sub-questions you must resolve (silently). Then answer.”

* **Refusal mode:**
  “If the question is outside {{scope}}, reply: ‘Sorry, I can only discuss {{scope}}.’”

* **Critic / Improve loop:**
  First pass: draft. Second pass: “Critique the draft against these criteria: … Now produce the improved final.”

* **Style-transfer:**
  “Rewrite the answer to match Tone=‘friendly, direct’, Sentences ≤ 2 lines.”

* **Self-consistency (sampling n):** run n>1 samples at T\~0.7 and pick majority/aggregated (in code orchestration).

---

## Context Engineering (RAG) Toolkit

### Chunking & Metadata

* **Chunk size:** 300–800 tokens (test 512 vs 1024).
* **Overlap:** 10–20% to preserve coherence.
* **Metadata:** title, section, date, source\_id, tags; store for filters & citations.

### Retrieval Strategies

* **Pure dense (embeddings):** cosine similarity on k=3–5.
* **Hybrid:** BM25 (keyword) + dense; merge scores.
* **Re-ranking:** retrieve k=20 dense → re-rank top 5 with a cross-encoder (optional).
* **Multi-vector / passage + summary:** store both raw chunks and their summaries.

### Context Packing

* **Stuffing:** paste top-k chunks. Simple, can overflow.
* **Refine:** start with answer from best chunk, refine with next chunks.
* **Map-Reduce:** summarize each chunk → combine summaries → answer.
* **Selective fields:** include only snippet, title, date, and a 1-line key point to save tokens.

### Anti-Hallucination Prompts

Add explicitly:

* “If information is missing or uncertain, reply exactly: ‘I don’t know.’”
* “Never infer dates/metrics not present in CONTEXT.”
* “Prefer quoting snippets with \[source\_id].”

### Recency/Validity Hints

* Include `last_updated` in context header.
* Prompt: “Prefer the most recent snippet when conflicting (by last\_updated).”

---

## Guardrails & Safety

* **Scope limiter:** define the domain you can talk about.
* **Banned behaviors:** e.g., “No medical/financial advice.”
* **Refusal template:** predictable sentence for out-of-scope.
* **Toxicity filter (pre/post):** screen inputs/outputs if needed (in code).
* **Regex/JSON schema validation:** reject malformed outputs; re-prompt with error.

---

## Parameter Tuning (defaults to start)

* **temperature:** 0.2 (↑ for creativity; ↓ for precision).
* **top\_p:** 1.0 (rarely needed if temperature tuned).
* **max\_tokens:** budget-aware (e.g., ≤ 300 for concise replies).
* **frequency/presence penalties:** small positive values to reduce repetition.

---

## Debugging Checklist

1. **Bad grounding?** Increase k, try hybrid retrieval, add reranker.
2. **Hallucinations?** Strengthen refusal text; add “no external facts”; reduce temperature.
3. **Off-style?** Add few-shot examples; tighten persona/tone rules.
4. **Too long?** Add hard length cap and “omit extras.”
5. **Invalid JSON?** Provide schema + example + tool-enforced validation with a retry loop.

---

## Evaluation & Iteration

### Mini Eval Set (keep it small but sharp)

* 10 in-scope direct Qs (answerable).
* 5 out-of-scope Qs (should refuse).
* 5 ambiguous/underspecified Qs (should ask clarifying or say “don’t know”).
* 5 adversarial (similar names, tricky dates).

### Metrics

* **Grounded accuracy:** correct & supported by context.
* **Refusal accuracy:** % of OOS correctly refused.
* **Format adherence:** valid JSON / style compliance.
* **Conciseness/readability:** manual rubric 1–5.
* **Latency & token usage.**

### Workflow

* Freeze a **Prompt vX**; run eval; log deltas.
* Change **one variable at a time** (k, chunk size, prompt copy, temperature).
* Keep a **prompt changelog** with rationales.

---

## Tips & Tricks (battle-tested)

* **Name the sections** in your prompt (ROLE, CONTEXT, TASK, RULES) → improves compliance.
* **Use exact strings** for refusals (“I don’t know.”) to simplify evals.
* **Give a target length** (words, bullets, or tokens), not “be concise”.
* **Pre-pend a TL;DR** summary to packed context when documents are long.
* **Order context by score + recency**; label each snippet `[S1]…` for citing.
* **Few-shot exemplars**: keep them **short** and structurally similar to desired outputs.
* **Deterministic JSON:** forbid extra keys; provide 1 valid example.
* **Hidden thoughts:** ask for careful internal reasoning but **only output final answer** (avoid printing chain-of-thought).
* **Version everything:** prompt text, k, chunking, reranker on/off.

---

## Ready-to-Use Mini Libraries (copy/paste)

### Short Answer (strict)

```
ROLE: Domain-bound assistant. Only use CONTEXT.

CONTEXT:
{{context}}

QUESTION:
{{question}}

REPLY STYLE:
- ≤ 5 sentences.
- Quote source IDs like [S#] when relevant.
- If not in CONTEXT: "I don’t know."
```

### Clarifying Questions First

```
If the QUESTION is ambiguous or missing key details, ask up to 2 clarifying questions before answering. 
If sufficient, answer normally; else say: "I don’t know."
```

### Summarizer (map-reduce compatible)

```
You will receive multiple snippets. For each snippet, produce a 1–2 sentence summary with its [S#].
Then combine into a coherent, non-redundant final summary, preserving source IDs.
```

### JSON Extractor

```
ROLE: Extract fields from CONTEXT. Output valid JSON only.

SCHEMA:
{"name":"","role":"","dates":[],"skills":[]}

RULES:
- If a field is missing, use empty string/array.
- Do not invent values.
```

---

## Example: Putting It Together (RAG Q\&A)

```
ROLE: You answer questions strictly from Harshit’s portfolio data.

CONTEXT (ranked, newest first):
[S1|2025-03-18] VGG-16 fine-tuned; best accuracy 64% …
[S2|2025-03-10] Using FER2013 dataset …
[S3|2024-11-26] Digit recognition project completed …

QUESTION:
What dataset did Harshit use for the emotion model?

RULES:
- Prefer newest when conflicting.
- Cite source IDs.
- If unknown: "I don’t know."
- ≤ 60 words.

EXPECTED OUTPUT STYLE:
"Harshit used the FER2013 dataset for the emotion classification project. [S2]"
```

---

## Practice Ladder (quick plan)

1. **Single-turn prompts**: zero → few-shot → persona → JSON.
2. **Context packing**: k=3 stuffing → refine → map-reduce → hybrid.
3. **Guardrails**: scope limiter + refusal tests.
4. **Eval & versioning**: lock a v1; iterate with tiny, visible changes.

---